# 'Living' Conclusion Gathering Space

## Conclusions Data Merge

-  There seems to be no pattern in the null values in target column. This indicates we can drop these rows


_County_
- we think county 12 is weird (unknown location) and introduces NA values (not included in weather data)
- maybe drop 12, but this may lead to other problems 
- county named 'unknown'
- the forums say counties 16 and 0 may be weird

_data_block_id_
- we could reduce NaN and NaT values by excluding data_block_id 1 and 0 (beginning in data set)

_Modelling / Time Series_
- We are unsure about modelling (is time series model needed? maybe ARMA?) Forums suggest e.g. XGBoost


## Conclusions EDA
- consumption has noticeable affects by winter holidays
- seems consumption is growing over time
- county 0 is dominating, Tallinn located there
- seems like temperature to production ratio changed last year
- product_type 2 attract producers with small installed capacity thus low production
- while product_type 3 attract the opposite cluster, producers with a lot of installed_capacity
- Surface solar radiation seems to have a stronger correlation with target than direct_solar
- There seems to be a 'split' around 6000 (unit?) daily mean target
- We expected more businesses in the top-producers

# Preparing Data

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px

In [ ]:
client = pd.read_csv('../data/client.csv')
client.head()

## Client Data

In [ ]:
# Datatype conversion
client.date = pd.to_datetime(client.date)
client.info()

In [ ]:
# categoricals?
display(client.product_type.unique())
display(client.is_business.unique())
display(client.county.unique())

In [ ]:
sns.scatterplot(x = client.date, y = client.data_block_id)

In [ ]:
display(min(client.date))
display(max(client.date))
display(client.data_block_id.nunique())

    First day is '2021-09-01 00:00:00', last day is '2023-05-29 00:00:00'. There are 636 unique days, and data_block_id corresponds to date. 

## Electricity Prices Data

In [ ]:
electricity_prices = pd.read_csv('../data/electricity_prices.csv')
electricity_prices.head()

In [ ]:
electricity_prices.info()

In [ ]:
electricity_prices.forecast_date = pd.to_datetime(electricity_prices.forecast_date)
electricity_prices.origin_date = pd.to_datetime(electricity_prices.origin_date)

electricity_prices.info()

In [ ]:
electricity_prices.nunique()

In [ ]:
electricity_prices.forecast_date.unique()

    Electricity price forecast are available for each hour of the day (637 days * 24 hours = 15286 unique datetimes) (for 637 days, one day more than client data; somewhere there are 2h missing)




## Forecast Weather Data

In [ ]:
forecast_weather = pd.read_csv('../data/forecast_weather.csv')

forecast_weather.head()

In [ ]:
forecast_weather.info()

In [ ]:
forecast_weather.origin_datetime = pd.to_datetime(forecast_weather.origin_datetime)
forecast_weather.forecast_datetime = pd.to_datetime(forecast_weather.forecast_datetime)

forecast_weather.info()

In [ ]:
forecast_weather.nunique()

In [ ]:
forecast_weather.groupby(['latitude', 'longitude']).nunique()

    There are 112 unique combinations of lat and long (unique weather stations). 
    So for each forecast_date, there are 112 observations (one from each station). 


In [ ]:
display(forecast_weather.hours_ahead.nunique())
display(forecast_weather.origin_datetime.nunique())
display(forecast_weather.groupby('origin_datetime').forecast_datetime.size()/112)

In [ ]:
forecast_weather.groupby(['origin_datetime', 'forecast_datetime']).count()

## Gas Prices Data

In [ ]:
gas_prices = pd.read_csv('../data/gas_prices.csv')

gas_prices.head()

In [ ]:
gas_prices.info()

In [ ]:
gas_prices.forecast_date = pd.to_datetime(gas_prices.forecast_date)
gas_prices.origin_date = pd.to_datetime(gas_prices.origin_date)

gas_prices.info()

In [ ]:
gas_prices.describe()

## Historical Weather Data

In [ ]:
historical_weather = pd.read_csv('../data/historical_weather.csv')

historical_weather.head()

In [ ]:
historical_weather.datetime = pd.to_datetime(historical_weather.datetime)

historical_weather.info()

In [ ]:
historical_weather.isnull().sum()

## Train Data & Checking for NULL values

In [ ]:
train = pd.read_csv('../data/train.csv')

train.head()

In [ ]:
train.datetime = pd.to_datetime(train.datetime, format='%Y-%m-%d %H:%M:%S')
train.info()

In [ ]:
train.head()

In [ ]:
train.datetime[1000]

In [ ]:
train.isnull().sum()

In [ ]:
train.groupby('is_consumption').agg({'target': lambda x: x.isnull().sum()})

In [ ]:
train[train.target.isnull()]

    There seems to be no pattern in the null values in target column. This indicates we can drop these rows

In [ ]:
weather_station_to_county_mapping = pd.read_csv('../data/weather_station_to_county_mapping.csv')

weather_station_to_county_mapping.head()

In [ ]:
weather_station_to_county_mapping.county.unique()

In [ ]:
weather_station_to_county_mapping.county_name.unique()

https://www.kaggle.com/code/fabiendaniel/mapping-locations-and-county-codes/notebook  for county codes
Here, they remove the 'maa' appendix from the county names. but is this really needed?

## Data Merging (now we merge everything to train)

In [ ]:
train.head()

### Merge Client

In [ ]:
client.head()

In [ ]:
print(client.size, train.size)

In [ ]:
# append '_client' to merged columns
client.columns = [f"{column}_client" if column not in ['data_block_id', 'county', 'is_business', 'product_type'] else column for column in client.columns]

In [ ]:
# merge train and client

merged_df = pd.merge(train, client, on=['data_block_id', 'county', 'is_business', 'product_type'], how='left')

merged_df.head()


there are a few/a lot null values, especially at the beginning and end of period


In [ ]:
# how many eic counts per data_block_id?
merged_df[merged_df.eic_count_client.isnull()].data_block_id.value_counts()

In [ ]:
# do dates in train and client overlap? 
print(set(client.date_client.dt.date) ^ set(train.datetime.dt.date))
print(set(train.data_block_id) ^ set(client.data_block_id))

In [ ]:
def print_block(dbid):
    display("TRAIN", train[train['data_block_id'] == dbid])
#     display("FORC WEATHER", forecast_weather[forecast_weather['data_block_id'] == dbid])
    display("CLIENT", client[client['data_block_id'] == dbid])
#     display("HIST WEATHER", historical_weather[historical_weather['data_block_id'] == dbid])
#     display("E PRICES", electricity_prices[electricity_prices['data_block_id'] == dbid])
#     display("G PRICES", gas_prices[gas_prices['data_block_id'] == dbid])

print_block(10)

There is a problem with NULL values after merging. one source are the start and end dates, but we don't know whats happening in between and whether this is problematic.
Maybe some client data is sporadically missing?

### Merge Gas Prices

In [ ]:
gas_prices.head()

In [ ]:
# append _gas_prices to columns
gas_prices.columns = [f"{column}_gas_prices" if column != 'data_block_id' else column for column in gas_prices.columns]

In [ ]:
# merge gas_prices

merged_df = pd.merge(merged_df, gas_prices, on=['data_block_id'], how='left')

merged_df.tail()


### Merge Electricity Prices

In [ ]:
# add time column for merging with electricity data
merged_df['time_of_day'] = merged_df['datetime'].dt.time
merged_df.head()

In [ ]:
# Merge electricity prices
# the prices are available hourly -> create new column with time 

electricity_prices['time_of_day'] = electricity_prices.forecast_date.dt.time

# append electricity_prices to column names
electricity_prices.columns = [f"{column}_electricity_prices" if column not in ['time_of_day','data_block_id'] else column for column in electricity_prices.columns]



### Merge Electricity Prices

In [ ]:
# merge electricity_prices

merged_df = pd.merge(merged_df, electricity_prices, on = ['data_block_id', 'time_of_day'], how='left')

merged_df.head()

### Merge Historical Weather

In [ ]:
# historic weather

historical_weather.head()

In [ ]:
# get county and county_name from weather_station_to_county_mapping (merge on latitude and longitude)

# round lat and long to avoid mismatching due to different accuracy
historical_weather.latitude = historical_weather.latitude.round(1)
historical_weather.longitude = historical_weather.longitude.round(1)

weather_station_to_county_mapping.latitude = weather_station_to_county_mapping.latitude.round(1)
weather_station_to_county_mapping.longitude = weather_station_to_county_mapping.longitude.round(1)


In [ ]:
# merge historical weather to get counties
merged_hist_weather = pd.merge(historical_weather, weather_station_to_county_mapping, on=['latitude', 'longitude'], how='left')
# get time of day
merged_hist_weather['time_of_day'] = merged_hist_weather['datetime'].dt.time

# aggregate by county and time (summarize weather stations for same county)
merged_hist_weather = merged_hist_weather.groupby(['county', 'time_of_day', 'datetime', 'data_block_id']).mean(numeric_only=True).reset_index()

# append _hist_weather to column names
merged_hist_weather.columns = [f"{column}_hist_weather" if column not in ['county', 'time_of_day','data_block_id'] else column for column in merged_hist_weather.columns]


merged_hist_weather.sample()


In [ ]:
# merge to merged_df
merged_df = pd.merge(merged_df, merged_hist_weather, on=['data_block_id', 'time_of_day', 'county'], how='left')

### Merge Forecast Weather

In [ ]:
forecast_weather.head()

In [ ]:
# forecast weather

#round lat and long
forecast_weather.latitude = forecast_weather.latitude.round(1)
forecast_weather.longitude = forecast_weather.longitude.round(1)

# merge to get counties
merged_forecast_weather = pd.merge(forecast_weather, weather_station_to_county_mapping, on=['latitude', 'longitude'], how='left')
# merged_forecast_weather['time_of_day'] = merged_forecast_weather.

# # aggregate for duplicate locations
merged_forecast_weather = merged_forecast_weather.groupby(['county', 'forecast_datetime', 'data_block_id']).mean(numeric_only=True).reset_index()

# append forecast_weather to column names
merged_forecast_weather.columns = [f"{column}_forecast_weather" if column not in ['county', 'forecast_datetime','data_block_id'] else column for column in merged_forecast_weather.columns]


merged_forecast_weather.sample()

In [ ]:
merged_df.datetime.dt.tz_localize('EET', ambiguous=True, nonexistent='shift_forward')[80000]

In [ ]:
merged_df.datetime.dt.tz_localize('EET', ambiguous=True, nonexistent='shift_forward')[1500000]

In [ ]:
merged_forecast_weather.forecast_datetime.dt.tz_convert('EET')[6000]

In [ ]:
# add EET timezone to datetime, and handle daylight-savings
merged_df['datetime_localized'] = merged_df.datetime.dt.tz_localize('EET', ambiguous=True, nonexistent='shift_forward')

# convert UTC timezone to EET timezone in forecast weather
merged_forecast_weather['datetime_EET']  = merged_forecast_weather.forecast_datetime.dt.tz_convert('EET')


In [ ]:
merged_forecast_weather.query('data_block_id == 300')

In [ ]:
# merge forecast_weather
merged_df = pd.merge(merged_df, merged_forecast_weather, left_on=['data_block_id', 'datetime_localized', 'county'], right_on=['data_block_id', 'datetime_EET', 'county'], how='left')

merged_df.sample(10)

In [ ]:
merged_df.info()

In [ ]:
merged_df.size

## Checking for NULL values on merged data

In [ ]:
merged_df.isnull().sum()

In [ ]:

merged_df.query('data_block_id != 0 and data_block_id != 1').isnull().sum()

In [ ]:
merged_df.query('data_block_id != 0 and data_block_id != 1')[merged_df.query('data_block_id != 0 and data_block_id != 1').isnull()].data_block_id

In [ ]:
#sns.scatterplot(data=merged_df, x='datetime', y= 'temperature_hist_weather', hue='county')

In [ ]:
import missingno as msno

In [ ]:
merged_df.head()

In [ ]:
msno.matrix(merged_df)

In [ ]:
msno.matrix(merged_df.query('data_block_id != 0 and data_block_id != 1'))

In [ ]:
msno.bar(merged_df)

In [ ]:
merged_df[(merged_df.temperature_hist_weather.isnull()) & (merged_df.data_block_id == 200)]

# .datetime.dt.date.unique()

In [ ]:
merged_df.groupby('county').aggregate(lambda x: x.isnull().sum())

In [ ]:
train.query('county == 11').shape

In [ ]:
weather_station_to_county_mapping.query('county == 12')

# EDA

In [ ]:
import seaborn as sns

In [ ]:
subsampled_df = merged_df.sample(10000)
#sns.scatterplot(data=merged_df, x='datetime', y = 'target', hue='is_consumption')

In [ ]:
sns.scatterplot(data=subsampled_df, x='datetime', y = 'target', hue='is_consumption')

In [ ]:
sns.kdeplot(data=subsampled_df, x = 'target', hue='is_consumption')


In [ ]:
# subset into production df

production_df = merged_df.query('is_consumption == 0').groupby('datetime').mean(numeric_only=True)

# we should aggregate target by sum, weather variables by  mean

production_df.head()

In [ ]:
production_df.size

In [ ]:
sns.scatterplot(data=production_df, x = 'cloudcover_total_forecast_weather', y = 'target')

In [ ]:
sns.scatterplot(data=production_df, x = 'direct_solar_radiation_forecast_weather', y = 'target')

In [ ]:
sns.scatterplot(data=production_df, x = 'surface_solar_radiation_downwards_forecast_weather', y = 'target', hue='is_business')

In [ ]:
production_df = merged_df.query('is_consumption == 0').sample(100000)
sns.scatterplot(data=production_df, x = 'surface_solar_radiation_downwards_forecast_weather', y = 'target', hue='product_type')

    - Surface solar radiation seems to have a stronger correlation with target than direct_solar
    - There seems to be a 'split' around 6000 (unit?) daily mean target
    - We expected more businesses in the top-producers


In [ ]:
client.describe()

### Focussing only on larger providers (is there a trend visible?)

In [ ]:
production_df = merged_df.query('(is_consumption == 0) and (1200 < installed_capacity_client < 1300)')
sns.scatterplot(data=production_df, x = 'surface_solar_radiation_downwards_forecast_weather', y = 'target', hue='product_type')


In [ ]:
merged_df.query('is_consumption == 0').groupby('product_type')['installed_capacity_client'].aggregate(['min', 'mean', 'max'])

    - product_type 2 attract producers with small installed capacity thus low production
    - while product_type 3 attract the opposite cluster, producers with a lot of installed_capacity

## Further quick Visualizations

In [ ]:
merged_df.plot(x='datetime', y='euros_per_mwh_electricity_prices')
plt.title('euros_per_mwh_electricity_prices')
plt.show()

In [ ]:
plt.plot(merged_df['datetime'], merged_df['lowest_price_per_mwh_gas_prices'], label = 'lowest price')
plt.plot(merged_df['datetime'], merged_df['highest_price_per_mwh_gas_prices'], label = 'highest price')
plt.title('highest and lowest price_per_mwh_gas_prices')
plt.legend()
plt.show()

In [ ]:
merged_df.plot(x='datetime', y='temperature_hist_weather')
plt.title('temperature_hist_weather')
plt.show()

In [ ]:
merged_df['dewpoint_hist_weather'].plot()
plt.title('dewpoint_hist_weather')
plt.show()

In [ ]:
merged_df['rain_hist_weather'].plot()
plt.title('rain_hist_weather')
plt.show()

In [ ]:
merged_df['snowfall_hist_weather'].plot()
plt.title('snowfall_hist_weather')
plt.show()

In [ ]:
merged_df['cloudcover_total_hist_weather'].plot()
plt.title('cloudcover_total_hist_weather')
plt.show()

In [ ]:
merged_df[['shortwave_radiation_hist_weather','direct_solar_radiation_hist_weather', 'diffuse_radiation_hist_weather']].plot()
plt.title('solar radiation')
plt.show()

In [ ]:
numeric_columns = merged_df.select_dtypes(include=[np.number])

correlation_matrix = numeric_columns.corr()

sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
# Create a 1x3 grid of subplots
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))

# Plot the first graph
merged_df.groupby('county')['target'].mean().plot(kind='bar', ax=axes[0])
axes[0].set_title('Average Consumption per County')
axes[0].set_xlabel('County')
axes[0].set_ylabel('Average Consumption')

# Plot the second graph
merged_df.groupby('product_type')['target'].mean().plot(kind='bar', ax=axes[1])
axes[1].set_title('Average Consumption per Product type')
axes[1].set_xlabel('Product type')
axes[1].set_ylabel('Average Consumption')

# Plot the third graph
merged_df.groupby('is_business')['target'].mean().plot(kind='bar', ax=axes[2])
axes[2].set_title('Average Consumption per Business')
axes[2].set_xlabel('Business or not')
axes[2].set_ylabel('Average Consumption')

# Adjust layout to prevent clipping of titles
plt.tight_layout()

# Show the combined plot
plt.show()

In [ ]:
def split_datetime(data, col="datetime"):
    # What columns are of type datetime?
    datetime_columns = data.select_dtypes(include='datetime64').columns
    
    for c in datetime_columns:
        print(f"Timezone for {c} is {data[c].dt.tz}")

    # Adding columns for date & time
    data['year']    = data[col].dt.year
    data['quarter'] = data[col].dt.quarter
    data['month']   = data[col].dt.month
    data['week']    = data[col].dt.isocalendar().week
    data['hour']    = data[col].dt.hour 

    data['day_of_year']  = data[col].dt.day_of_year
    data['day_of_month'] = data[col].dt.day
    data['day_of_week']  = data[col].dt.day_of_week

    return data

# calculate sum of production of business producers and the average over the day of the week
business_production_sum_timeseries = merged_df.query('is_consumption == 0 & is_business == 1')[["datetime", "target"]].groupby("datetime").sum()
business_production_sum_timeseries.reset_index(inplace=True)
business_production_sum_timeseries = split_datetime(business_production_sum_timeseries, "datetime")
# Have a look at the production average per day of the week
business_production_av_day_of_week = business_production_sum_timeseries.groupby(["day_of_week", "hour"])[["target"]].mean()
business_production_av_day_of_week.reset_index(inplace=True)

# calculate sum of production of non-business producers and the average over the day of the week
non_business_production_sum_timeseries = merged_df.query('is_consumption == 0 & is_business == 0')[["datetime", "target"]].groupby("datetime").sum()
non_business_production_sum_timeseries.reset_index(inplace=True)
non_business_production_sum_timeseries = split_datetime(non_business_production_sum_timeseries, "datetime")
# Have a look at the production average per day of the week
non_business_production_av_day_of_week = non_business_production_sum_timeseries.groupby(["day_of_week", "hour"])[["target"]].mean()
non_business_production_av_day_of_week.reset_index(inplace=True)

plt.figure(figsize=(12,8))
plt.plot(business_production_av_day_of_week.index, business_production_av_day_of_week["target"], label="business")
plt.plot(non_business_production_av_day_of_week.index, non_business_production_av_day_of_week["target"], label="non-business")
plt.legend()

    Production for businesses is higher on the weekends

In [ ]:
# calculate sum of consumption of business producers and the average over the day of the week
business_consumption_sum_timeseries = merged_df.query('is_consumption == 1 & is_business == 1')[["datetime", "target"]].groupby("datetime").sum()
business_consumption_sum_timeseries.reset_index(inplace=True)
business_consumption_sum_timeseries = split_datetime(business_consumption_sum_timeseries, "datetime")
# Have a look at the consumption average per day of the week
business_consumption_av_day_of_week = business_consumption_sum_timeseries.groupby(["day_of_week", "hour"])[["target"]].mean()
business_consumption_av_day_of_week.reset_index(inplace=True)

# calculate sum of consumption of business producers and the average over the day of the week
non_business_consumption_sum_timeseries = merged_df.query('is_consumption == 1 & is_business == 0')[["datetime", "target"]].groupby("datetime").sum()
non_business_consumption_sum_timeseries.reset_index(inplace=True)
non_business_consumption_sum_timeseries = split_datetime(non_business_consumption_sum_timeseries, "datetime")
# Have a look at the consumption average per day of the week
non_business_consumption_av_day_of_week = non_business_consumption_sum_timeseries.groupby(["day_of_week", "hour"])[["target"]].mean()
non_business_consumption_av_day_of_week.reset_index(inplace=True)

plt.figure(figsize=(12,8))
plt.plot(business_consumption_av_day_of_week.index, business_consumption_av_day_of_week["target"], label="business")
plt.plot(non_business_consumption_av_day_of_week.index, non_business_consumption_av_day_of_week["target"], label="non-business")
plt.legend()

In [ ]:
small_producers = merged_df.query('is_consumption == 0 & installed_capacity_client < 1000')
small_producers_sum_timeseries = small_producers[["datetime", "county", "target", "surface_solar_radiation_downwards_forecast_weather", "installed_capacity_client"]].groupby(["county", "datetime"]).agg({"target": "sum", "surface_solar_radiation_downwards_forecast_weather": "mean", "installed_capacity_client": "mean"})
small_producers_sum_timeseries.reset_index(inplace=True)

plt.figure(figsize=(12,8))
sns.scatterplot(data=small_producers_sum_timeseries.query("county == 0"), x="surface_solar_radiation_downwards_forecast_weather", y="target", hue="installed_capacity_client")

large_producers = merged_df.query('is_consumption == 0 & installed_capacity_client >= 1000')
large_producers_sum_timeseries = large_producers[["datetime", "county", "target", "surface_solar_radiation_downwards_forecast_weather", "installed_capacity_client"]].groupby(["county", "datetime"]).agg({"target": "sum", "surface_solar_radiation_downwards_forecast_weather": "mean", "installed_capacity_client": "mean"})
large_producers_sum_timeseries.reset_index(inplace=True)

plt.figure(figsize=(12,8))
sns.scatterplot(data=large_producers_sum_timeseries.query("county == 0"), x="surface_solar_radiation_downwards_forecast_weather", y="target", hue="installed_capacity_client")

    Solar radiation and production seems to be linear, but the line differs from how much capacity you have (more or less two lines running from the same origin, like a "star").

In [ ]:
px.line(merged_df.groupby(['county', 'datetime', 'is_consumption']).mean(numeric_only=True).reset_index(),
         x='datetime', y='target', color='county', facet_col='is_consumption')

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# trace 0 = surface solar radat
# trace 1 = target
# trace 2 = sum of installed capacity

#df = merged_df.query('(is_business not in [1]) and (is_consumption == 0)').groupby(['datetime']).mean(numeric_only=True).reset_index()
aggregations = {'target': 'sum', 'surface_solar_radiation_downwards_forecast_weather': 'mean', 'installed_capacity_client': 'sum'}
try:
    df = merged_df.query('(is_business not in [1]) and (is_consumption == 0)').groupby(['datetime']).aggregate(aggregations).reset_index()
except:
    print('mean applied')
    df = merged_df.query('(is_business not in [1]) and (is_consumption == 0)').groupby(['datetime']).aggregate('mean').reset_index()


fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=df['datetime'], y=df['surface_solar_radiation_downwards_forecast_weather'], opacity=0.7),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=df['datetime'], y=df['target'], opacity=0.7),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df['datetime'], y=df['installed_capacity_client'], opacity=0.7),
    secondary_y=False,
)

fig

In [ ]:
merged_df.head()

# Hypotheses and baseline model ideas

- Simple Baseline Ideas
  - **Simple baseline model**: Average (average) consumption of day before this year would prediction consumption for next year
  - How is **weather of day before** is influencing production and consumption (if sunshine day before is low, production next day would possibly be lower)

  - **Very simple regression model**, with few features (solar radiation colors, snow/temperature, capacity)
- Correlation
  - What are parameters for the **highest correlation for consumption** (e.g. higher solar radiation, the higher the production; the higher the capacity, the higher the production)
  - Gas prices and electricity prices change -> correlation
- **Two models**: production (solar and capacity), consumption (temperature); target represents if consumption or production


# Feature engineering

### Add columns for date / time

In [ ]:
def add_features(data):
    # What columns are of type datetime?
    datetime_columns = merged_df.select_dtypes(include='datetime64').columns
    
    for c in datetime_columns:
        print(f"Timezone for {c} is {data[c].dt.tz}")

    # Adding columns for date & time
    data['year']    = data['datetime'].dt.year
    data['quarter'] = data['datetime'].dt.quarter
    data['month']   = data['datetime'].dt.month
    data['week']    = data['datetime'].dt.isocalendar().week
    data['hour']    = data['datetime'].dt.hour 

    data['day_of_year']  = data['datetime'].dt.day_of_year
    data['day_of_month'] = data['datetime'].dt.day
    data['day_of_week']  = data['datetime'].dt.day_of_week

    return data

merged_df = add_features(merged_df)

## -> need to convert to EET timezone

## Baseline model


model ideas
- calculate average of one year, use this as a prediction for next year ? (but we know already that seasonality is important)
- ** prediction of t is equal to t-1year  **


to do
- define x and y, only 'model' on most recent year (2023?)




In [ ]:
test = train.copy()

test.head()

In [ ]:
# create Baseline Model

test_offset = train.copy()
test_offset
test_offset['datetime'] = test_offset['datetime'] + pd.Timedelta(value=365, unit='days')  # PLUS or MINUS?

test_offset.head()

In [ ]:
# merge df with baseline predictions (in test_offset)
test = test.merge(
    test_offset, 
    on=['county', 'is_business', 'product_type', 'is_consumption', 'datetime'], 
    how='left', 
    suffixes=('', '_previous_year')
)

In [ ]:
# idea: define function to pull target from 1 year ago

'''def baseline(date):
    prediction  = target from one year ago   

    return prediction'''

In [ ]:
from sklearn.metrics import mean_absolute_error

test.dropna(inplace=True)

print('Previous year as prediction:', mean_absolute_error(test['target'], test['target_previous_year']))


In [ ]:
# define error fun
def calc_mae(x, y):
    mae = np.mean(np.abs(y - x))
    return mae

In [ ]:
# generate df for plotting
plotdat = test.groupby(['datetime', 'is_consumption']).mean().reset_index()

In [ ]:
print('MAE consumption: ', calc_mae(plotdat.query('is_consumption == 1')['target'], plotdat.query('is_consumption == 1')['target_previous_year']))
print('MAE production:  ', calc_mae(plotdat.query('is_consumption == 0')['target'], plotdat.query('is_consumption == 0')['target_previous_year']))

print('MAE all:         ', calc_mae(plotdat['target'], plotdat['target_previous_year']))

Results of pre-implemented sklearn MAE are different from the defined calc_mae function. It is unclear why, but Kaggle uses the pre-implemented function -> this is what we should optimize for. 

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

test_prod = test.query('is_consumption == 0')

fig.add_trace(
    go.Scatter(x=test_prod['datetime'], y=test_prod['target'], opacity=0.7, name='production_true'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=test_prod['datetime'], y=test_prod['target_previous_year'], opacity=0.7, name='production_pred'),
    secondary_y=False,
)

fig

In [ ]:
# plot actual and baseline-predicted consumption

plt.plot(plotdat.query('is_consumption == 1')['datetime'], plotdat.query('is_consumption == 1')['target'])
plt.plot(plotdat.query('is_consumption == 1')['datetime'], plotdat.query('is_consumption == 1')['target_previous_year'], 'r')

plt.legend(['obs.', 'pred.'])
plt.title('Baseline model: observed and predicted consumption')

In [ ]:
# plot actual and baseline-predicted production

plt.plot(plotdat.query('is_consumption == 0')['datetime'], plotdat.query('is_consumption == 0')['target'])
plt.plot(plotdat.query('is_consumption == 0')['datetime'], plotdat.query('is_consumption == 0')['target_previous_year'], 'r')

plt.legend(['obs.', 'pred.'])
plt.title('Baseline model: observed and predicted production')

In [ ]:
# plot actual and baseline-predicted production
# average over data block id, because during night (no sun), production is 0 and therefore plot becomes unreadable

plotdat2 = plotdat.groupby(['data_block_id', 'is_consumption']).mean().reset_index()

plt.plot(plotdat2.query('is_consumption == 0')['datetime'], plotdat2.query('is_consumption == 0')['target'])
plt.plot(plotdat2.query('is_consumption == 0')['datetime'], plotdat2.query('is_consumption == 0')['target_previous_year'], 'r')

plt.legend(['obs.', 'pred.'])
plt.title('Baseline model: observed and predicted production')

# Modelling

In [ ]:
del train
del client
del gas_prices
del electricity_prices
del forecast_weather
del historical_weather

In [ ]:
# copy df for modelling
model_df = merged_df.copy()

# model is not able to handle object type
model_df.drop('time_of_day', axis=1, inplace=True)

# split datetime into meaningful features of int types
model_df = split_datetime(model_df)

# model is not able to handle datetime
model_df = model_df.drop(model_df.select_dtypes(include=['datetime64[ns]', 'datetime64[ns, EET]']).columns, axis=1)

# drop na from target
model_df.dropna(subset=['target'], inplace=True)


In [ ]:
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train,  y_test = train_test_split(model_df.drop('target', axis=1), model_df['target'], test_size=0.3, random_state=0)

bst = XGBRegressor(enable_categorical=True)
bst.fit(X_train, y_train)
y_pred = bst.predict(X_test)

# main optimisation metric
print('Mean absolute error test', mean_absolute_error(y_test, y_pred))
print('Mean absolute error train', mean_absolute_error(y_train, bst.predict(X_train)))

In [ ]:
# first attempt gave us 50.75 mean absolute error

In [ ]:
model_df['data_block_id'].describe()

In [ ]:
# split of old data to train and newer one to test

Xy_train = model_df[model_df.data_block_id < 450]
X_train = Xy_train.drop('target', axis=1)
y_train = Xy_train.target

Xy_test = model_df[model_df.data_block_id >= 450]
X_test = Xy_test.drop('target', axis=1)
y_test = Xy_test.target

bst = XGBRegressor(enable_categorical=True)
bst.fit(X_train, y_train)
y_pred = bst.predict(X_test)

# main optimisation metric
print('Mean absolute error test', mean_absolute_error(y_test, y_pred))
print('Mean absolute error train', mean_absolute_error(y_train, bst.predict(X_train)))

In [ ]:
# divide by dates and use newer ones for validation

In [ ]:
bst.feature_names_in_

In [ ]:
bst.feature_importances_

In [ ]:
features_dict = {key: value for key, value in zip(bst.feature_names_in_, bst.feature_importances_)}
features_dict

In [ ]:
xgb.plot_importance(bst)
plt.title('Feature Importance')
plt.show()

In [ ]:
# hours_ahead_forecast treated as important feature, probably smth to drop)

- visualisation
- split by date
- tweaking the parameters
- drop some features
- feature engineering
- overfitting with traditional train_test_split?
- try to models/ multiple_output/ other models

In [ ]:
# model_df2 = model_df.copy()
# model_df2.drop(['row_id', ])

split_datablock = 300

Xy_train = model_df[model_df.data_block_id < split_datablock]
X_train = Xy_train.drop('target', axis=1)
y_train = Xy_train.target

Xy_test = model_df[model_df.data_block_id >= split_datablock]
X_test = Xy_test.drop('target', axis=1)
y_test = Xy_test.target

bst = XGBRegressor(enable_categorical=True)
bst.fit(X_train, y_train)

y_pred_test = bst.predict(X_test)
y_pred_train = bst.predict(X_train)

# main optimisation metric
print('Mean absolute error test', mean_absolute_error(y_test, y_pred_test))
print('Mean absolute error train', mean_absolute_error(y_train, y_pred_train))

In [ ]:
px.scatter(x=Xy_train.index, y=y_pred_train-y_train, color=Xy_train.month)

In [ ]:
px.scatter(data_frame=Xy_test, x=Xy_test.index, y=y_pred_test-y_test, color=Xy_test.month, hover_data='day_of_week')

In [ ]:
Xy_test['residual'] = y_pred_test-y_test

Xy_test.head()

In [ ]:
sns.heatmap(Xy_test.corr(numeric_only=True), annot=False, cmap='RdBu', center = 0)
plt.title('Correlation Heatmap')
plt.figure(
    figsize=(20, 20)
)
plt.show()

In [ ]:
target_column = 'residual'

# Exclude non-numeric columns
numeric_columns = Xy_test.select_dtypes(include=['number']).columns
numeric_df = Xy_test[numeric_columns]
numeric_df_cons = numeric_df[numeric_df['is_consumption'] == 1]

# Calculate the correlation matrix
correlation_matrix = numeric_df_cons.corr()

# Select correlations based on the threshold
threshold = 0.15
significant_correlations = correlation_matrix[(correlation_matrix[target_column] > threshold) | (correlation_matrix[target_column] < -threshold)][target_column]

# Plot a heatmap of the significant correlations with the target
plt.figure(figsize=(12, 8))
sns.heatmap(significant_correlations.to_frame(), annot=True, cmap='coolwarm', fmt=".2f", cbar=False)
plt.title(f'Significant Correlations with {target_column}, CONSUM ONLY (Threshold: {threshold})')
plt.show()

In [ ]:
target_column = 'residual'

# Exclude non-numeric columns
numeric_columns = Xy_test.select_dtypes(include=['number']).columns
numeric_df = Xy_test[numeric_columns]
numeric_df_cons = numeric_df[numeric_df['is_consumption'] == 0]

# Calculate the correlation matrix
correlation_matrix = numeric_df_cons.corr()

# Select correlations based on the threshold
threshold = 0.15
significant_correlations = correlation_matrix[(correlation_matrix[target_column] > threshold) | (correlation_matrix[target_column] < -threshold)][target_column]

# Plot a heatmap of the significant correlations with the target
plt.figure(figsize=(12, 8))
sns.heatmap(significant_correlations.to_frame(), annot=True, cmap='coolwarm', fmt=".2f", cbar=False)
plt.title(f'Significant Correlations with {target_column}, PRODUCTION ONLY (Threshold: {threshold})')
plt.show()

- residuals are bigger at the summer time, we quess because production is happening at this time

- residuals on the test data have weekly pattern
- last two month predicted very poorly

- residuals are different depending on how we split our data 
- we see unexpalinable patterns in residuals
- residuals for consumption and production correlate with different features

- try residual analysis with traditional test_train_split

- tweak the model

In [ ]:
X_train, X_test, y_train,  y_test = train_test_split(model_df.drop('target', axis=1), model_df['target'], test_size=0.3, random_state=0)

bst = XGBRegressor(enable_categorical=True)
bst.fit(X_train, y_train)
y_pred = bst.predict(X_test)

# main optimisation metric
print('Mean absolute error test', mean_absolute_error(y_test, y_pred))
print('Mean absolute error train', mean_absolute_error(y_train, bst.predict(X_train)))

y_pred_test = bst.predict(X_test)
y_pred_train = bst.predict(X_train)



In [ ]:
px.scatter(x=X_train.index, y=y_pred_train-y_train, color=X_train.month)

In [ ]:
px.scatter(data_frame=X_test, x=X_test.index, y=y_pred_test-y_test, color=X_test.month, hover_data='day_of_week')

- pretty much the same patterns with different train test splits
